### Data Generation

This notebook generates the training and validation data for training a model for predicting housing rental prices

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder
from src.cleaning import *
from src.preprocessing import *
from src.feature_eng import *
from src.utils import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
RAW_DATAFILE_TRAIN = "data/train.csv"
RAW_DATAFILE_TEST = "data/test.csv"

In [24]:
raw_df = pd.read_csv(RAW_DATAFILE_TRAIN)
raw_train_df, raw_val_df = train_test_split(raw_df, test_size=0.1, random_state=42)

In [25]:
raw_test_df = pd.read_csv(RAW_DATAFILE_TEST)

#### Data Cleaning
In this step, we perform:
- standardisation of strings to lowercase
- standardisation of flat type labels
- remove single-valued columns

In [26]:
# Base class for custom preprocessing steps
class BasePreprocessor:
    def __init__(self):
        self._train_features = []
        self._label_name = "monthly_rent"

    def fit_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self._preprocess(df)
        self._train_features = list(df.columns)
        self._train_features.remove(self._label_name)
        df = self._postprocess(df)
        return df

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self._preprocess(df)
        df = self._postprocess(df)
        return df

    def _preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        # Main function to preprocess cleaned data in the class
        df = self._clean_data(df)
        return df

    def _postprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        # Fix any missing/mismatch feature due to one hot encoding
        missing_feats = set(self._train_features) - set(df.columns)
        for feat in missing_feats:
            df[feat] = 0
        if self._label_name in df.columns:
            # train/val dataset
            df = df[self._train_features + [self._label_name]]
        else:
            # test dataset
            df = df[self._train_features]
        return df

    def _clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        # Clean the data (common step for all preprocessors)
        df = convert_strings_to_lowercase(df)
        df = clean_flat_type_labels(df)
        df = df.drop(columns=["furnished", "elevation", "street_name"])
        return df

### Dataset 1: Baseline
This is a baseline dataset with only minimal data cleaning and preprocessing done. It is used to train a baseline model with which we compare all other trained models.

#### Data Preprocessing
In this step, we perform the following preprocessing steps:
- convert `flat_type` to numerical form
- reduce the number of categories of `flat_model`
- perform variable scaling to derive remaining lease period on `lease_commence_date`
- perform one-hot-encoding on categorical variables
- split `rent_approval_date` into month and year columns
- convert `block` to integer form, removing any letters

In [27]:
class BaselinePreprocessor(BasePreprocessor):
    def _preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self._clean_data(df)
        df = self._preprocess_baseline(df)
        return df

    def _preprocess_baseline(self, df: pd.DataFrame) -> pd.DataFrame:
        df['std_flat_type'] = df.apply(lambda x: std_flat_type(x['flat_type'], x['lease_commence_date']), axis=1)
        df['std_flat_model'] = df.apply(lambda x: std_flat_model(x['flat_model']), axis=1)
        df['std_remaining_lease'] = df.apply(lambda x: std_remaining_lease(x['lease_commence_date'], 2023), axis=1)
        df = preprocess_region(df)
        df = preprocess_planning_area(df)
        df = preprocess_flat_model(df)
        df = preprocess_town(df)
        df = preprocess_subzone(df)
        df = preprocess_rent_approval_date(df)
        df['block_num_int'] = df.apply(lambda x: preprocess_block(x['block']), axis=1)
        df = df.drop(columns=['block', 'flat_type', 'lease_commence_date'])
        return df

In [28]:
baseline_preprocessor = BaselinePreprocessor()
baseline_train = baseline_preprocessor.fit_transform(raw_train_df)

baseline_val = baseline_preprocessor.transform(raw_val_df)
baseline_test = baseline_preprocessor.transform(raw_test_df)

In [29]:
baseline_train.to_csv("data/train/baseline_train.csv", index=False)
baseline_val.to_csv("data/train/baseline_val.csv", index=False)
baseline_test.to_csv("data/test/baseline_test.csv", index=False)

In [30]:
baseline_train.head()

,floor_area_sqm,latitude,longitude,std_flat_type,std_remaining_lease,central region,east region,north region,north-east region,west region,planning_area_ ang mo kio,planning_area_ bedok,planning_area_ bishan,planning_area_ bukit batok,planning_area_ bukit merah,planning_area_ bukit panjang,planning_area_ bukit timah,planning_area_ choa chu kang,planning_area_ clementi,planning_area_ downtown core,planning_area_ geylang,planning_area_ hougang,planning_area_ jurong east,planning_area_ jurong west,planning_area_ kallang,planning_area_ marine parade,planning_area_ novena,planning_area_ outram,planning_area_ pasir ris,planning_area_ punggol,planning_area_ queenstown,planning_area_ rochor,planning_area_ sembawang,planning_area_ sengkang,planning_area_ serangoon,planning_area_ tampines,planning_area_ toa payoh,planning_area_ woodlands,planning_area_ yishun,A,I,NG,OTH,P,S,STD,ang mo kio,bedok,bishan,bukit batok,bukit merah,bukit panjang,bukit timah,central,choa chu kang,clementi,geylang,hougang,jurong east,jurong west,kallang/whampoa,marine parade,pasir ris,punggol,queenstown,sembawang,sengkang,serangoon,tampines,toa payoh,woodlands,yishun,subzone admiralty,subzone alexandra hill,subzone aljunied,subzone anak bukit,subzone anchorvale,subzone ang mo kio town centre,subzone balestier,subzone bangkit,subzone bedok north,subzone bedok reservoir,subzone bedok south,subzone bencoolen,subzone bendemeer,subzone bishan east,subzone boon keng,subzone boon lay place,subzone boon teck,subzone braddell,subzone brickworks,subzone bugis,subzone bukit batok central,subzone bukit batok east,subzone bukit batok south,subzone bukit batok west,subzone bukit ho swee,subzone bukit merah,subzone cheng san,subzone china square,subzone chinatown,subzone choa chu kang central,subzone choa chu kang north,subzone chong boon,subzone city hall,subzone clementi central,subzone clementi north,subzone clementi west,subzone clementi woods,subzone commonwealth,subzone compassvale,subzone crawford,subzone depot road,subzone dover,subzone everton park,subzone fajar,subzone farrer park,subzone fernvale,subzone frankel,subzone geylang bahru,subzone geylang east,subzone ghim moh,subzone gombak,subzone guilin,subzone henderson hill,subzone holland drive,subzone hong kah,subzone hong kah north,subzone hougang central,subzone hougang east,subzone hougang west,subzone jelebu,subzone joo seng,subzone jurong west central,subzone kaki bukit,subzone kampong bugis,subzone kampong java,subzone kampong tiong bahru,subzone kampong ubi,subzone kangkar,subzone keat hong,subzone kebun bahru,subzone kembangan,subzone khatib,subzone kim keat,subzone kovan,subzone lavender,subzone little india,subzone lorong 8 toa payoh,subzone lorong ah soo,subzone lower seletar,subzone macpherson,subzone margaret drive,subzone marine parade,subzone marymount,subzone matilda,subzone mei chin,subzone midview,subzone moulmein,subzone north coast,subzone northland,subzone pasir panjang 2,subzone pasir ris central,subzone pasir ris drive,subzone pasir ris west,subzone pearl's hill,subzone pei chun,subzone peng siang,subzone potong pasir,subzone punggol field,subzone punggol town centre,subzone redhill,subzone rivervale,subzone saujana,subzone sembawang central,subzone sembawang east,subzone sembawang north,subzone sengkang town centre,subzone senja,subzone serangoon central,subzone serangoon garden,subzone serangoon north,subzone shangri-la,subzone simei,subzone sungei road,subzone sunset way,subzone swiss club,subzone tai seng,subzone taman jurong,subzone tampines east,subzone tampines west,subzone tanglin halt,subzone tanjong rhu,subzone teban gardens,subzone teck whye,subzone telok blangah drive,subzone telok blangah rise,subzone telok blangah way,subzone tiong bahru,subzone tiong bahru station,subzone toa payoh central,subzone toa payoh west,subzone toh guan,subzone townsville,subzone trafalgar,subzone ulu pandan,subzone upper paya lebar,subzone upper thomson,subzone victoria,subzone waterway east,subzone w

#### Choice between `monthly_rent` and `rent_per_sqm` label 

For this dataset (and other datasets below), the training can be chosen between `monthly_rent` (original labels) or `rent_per_sqm` label. This can be done by using the functions from `src/utils.py`

`monthly_rent` label:

In [31]:
# training
X_train, y_train = split_features_and_monthly_rent_label(baseline_train)
X_val, y_val = split_features_and_monthly_rent_label(baseline_val)

# testing
X_test, _ = split_features_and_monthly_rent_label(baseline_test)

`rent_per_sqm` label:

In [32]:
# training
X_train, y_train, floor_area_sqm_train = split_features_and_rent_per_sqm_label(baseline_train)
X_val, y_val, floor_area_sqm_val = split_features_and_rent_per_sqm_label(baseline_val)

# testing
X_test, _, floor_area_sqm_test = split_features_and_rent_per_sqm_label(baseline_test)
# X_test -> y_rent_per_sqm_test

# after prediction, convert label back to `monthly_rent` for true prediction and evaluation
y_rent_per_sqm_test = np.ones(shape=(X_test.shape[0],))  # dummy prediction
y_monthly_rent_test = convert_rent_per_sqm_label_to_monthly_rent_label(y_rent_per_sqm_test, floor_area_sqm_test)

### Dataset 2: Dataset with Additional Features
In this dataset, additional features below are added based on our initial data analysis
- Mean monthly COE prices
- Number of amenities near a house

In [33]:
class FeatEngPreprocessor(BasePreprocessor):
    def __init__(self):
        super().__init__()
        self._coe_prices = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-coe-prices.csv")
        self._amenities_df = pd.read_csv("data/auxiliary-data/auxiliary-data/amenity_data_allmrt.csv")

    def _preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self._clean_data(df)
        df = self._preprocess_baseline(df)
        df = self._preprocess_feature_eng(df)
        return df

    def _preprocess_baseline(self, df: pd.DataFrame) -> pd.DataFrame:
        df['std_flat_type'] = df.apply(lambda x: std_flat_type(x['flat_type'], x['lease_commence_date']), axis=1)
        df['std_flat_model'] = df.apply(lambda x: std_flat_model(x['flat_model']), axis=1)
        df['std_remaining_lease'] = df.apply(lambda x: std_remaining_lease(x['lease_commence_date'], 2023), axis=1)
        df = preprocess_region(df)
        df = preprocess_planning_area(df)
        df = preprocess_flat_model(df)
        df = preprocess_town(df)
        df = preprocess_subzone(df)
        df = preprocess_rent_approval_date(df)
        df['block_num_int'] = df.apply(lambda x: preprocess_block(x['block']), axis=1)
        df = df.drop(columns=['block', 'flat_type', 'lease_commence_date'])
        return df

    def _preprocess_feature_eng(self, df: pd.DataFrame) -> pd.DataFrame:
        _, df = compute_mean_coe_prices(self._coe_prices.copy(), df)
        df = count_amenity(df, self._amenities_df.copy(), 0.8)
        return df

In [34]:
feature_eng_preprocessor = FeatEngPreprocessor()
feature_eng_train = feature_eng_preprocessor.fit_transform(raw_train_df)

feature_eng_val = feature_eng_preprocessor.transform(raw_val_df)
feature_eng_test = feature_eng_preprocessor.transform(raw_test_df)

In [35]:
feature_eng_train.to_csv("data/train/baseline-w-feature-eng_train.csv", index=False)
feature_eng_val.to_csv("data/train/baseline-w-feature-eng_val.csv", index=False)
feature_eng_test.to_csv("data/test/baseline-w-feature-eng_test.csv", index=False)

In [36]:
feature_eng_train.head()

,floor_area_sqm,latitude,longitude,std_flat_type,std_remaining_lease,central region,east region,north region,north-east region,west region,planning_area_ ang mo kio,planning_area_ bedok,planning_area_ bishan,planning_area_ bukit batok,planning_area_ bukit merah,planning_area_ bukit panjang,planning_area_ bukit timah,planning_area_ choa chu kang,planning_area_ clementi,planning_area_ downtown core,planning_area_ geylang,planning_area_ hougang,planning_area_ jurong east,planning_area_ jurong west,planning_area_ kallang,planning_area_ marine parade,planning_area_ novena,planning_area_ outram,planning_area_ pasir ris,planning_area_ punggol,planning_area_ queenstown,planning_area_ rochor,planning_area_ sembawang,planning_area_ sengkang,planning_area_ serangoon,planning_area_ tampines,planning_area_ toa payoh,planning_area_ woodlands,planning_area_ yishun,A,I,NG,OTH,P,S,STD,ang mo kio,bedok,bishan,bukit batok,bukit merah,bukit panjang,bukit timah,central,choa chu kang,clementi,geylang,hougang,jurong east,jurong west,kallang/whampoa,marine parade,pasir ris,punggol,queenstown,sembawang,sengkang,serangoon,tampines,toa payoh,woodlands,yishun,subzone admiralty,subzone alexandra hill,subzone aljunied,subzone anak bukit,subzone anchorvale,subzone ang mo kio town centre,subzone balestier,subzone bangkit,subzone bedok north,subzone bedok reservoir,subzone bedok south,subzone bencoolen,subzone bendemeer,subzone bishan east,subzone boon keng,subzone boon lay place,subzone boon teck,subzone braddell,subzone brickworks,subzone bugis,subzone bukit batok central,subzone bukit batok east,subzone bukit batok south,subzone bukit batok west,subzone bukit ho swee,subzone bukit merah,subzone cheng san,subzone china square,subzone chinatown,subzone choa chu kang central,subzone choa chu kang north,subzone chong boon,subzone city hall,subzone clementi central,subzone clementi north,subzone clementi west,subzone clementi woods,subzone commonwealth,subzone compassvale,subzone crawford,subzone depot road,subzone dover,subzone everton park,subzone fajar,subzone farrer park,subzone fernvale,subzone frankel,subzone geylang bahru,subzone geylang east,subzone ghim moh,subzone gombak,subzone guilin,subzone henderson hill,subzone holland drive,subzone hong kah,subzone hong kah north,subzone hougang central,subzone hougang east,subzone hougang west,subzone jelebu,subzone joo seng,subzone jurong west central,subzone kaki bukit,subzone kampong bugis,subzone kampong java,subzone kampong tiong bahru,subzone kampong ubi,subzone kangkar,subzone keat hong,subzone kebun bahru,subzone kembangan,subzone khatib,subzone kim keat,subzone kovan,subzone lavender,subzone little india,subzone lorong 8 toa payoh,subzone lorong ah soo,subzone lower seletar,subzone macpherson,subzone margaret drive,subzone marine parade,subzone marymount,subzone matilda,subzone mei chin,subzone midview,subzone moulmein,subzone north coast,subzone northland,subzone pasir panjang 2,subzone pasir ris central,subzone pasir ris drive,subzone pasir ris west,subzone pearl's hill,subzone pei chun,subzone peng siang,subzone potong pasir,subzone punggol field,subzone punggol town centre,subzone redhill,subzone rivervale,subzone saujana,subzone sembawang central,subzone sembawang east,subzone sembawang north,subzone sengkang town centre,subzone senja,subzone serangoon central,subzone serangoon garden,subzone serangoon north,subzone shangri-la,subzone simei,subzone sungei road,subzone sunset way,subzone swiss club,subzone tai seng,subzone taman jurong,subzone tampines east,subzone tampines west,subzone tanglin halt,subzone tanjong rhu,subzone teban gardens,subzone teck whye,subzone telok blangah drive,subzone telok blangah rise,subzone telok blangah way,subzone tiong bahru,subzone tiong bahru station,subzone toa payoh central,subzone toa payoh west,subzone toh guan,subzone townsville,subzone trafalgar,subzone ulu pandan,subzone upper paya lebar,subzone upper thomson,subzone victoria,subzone waterway east,subzone w

### Dataset 3: Baseline with Extra Columns Removed
This is a truncated version of the baseline dataset with the following columns removed to check if their inclusion/exclusion has any impact on predicted rental prices.
- block
- town
- subzone
- street_name


In [37]:
class BaselineTruncatedPreprocessor(BasePreprocessor):
    def _preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self._clean_data(df)
        df = self._preprocess_baseline_truncate(df)
        return df

    def _preprocess_baseline_truncate(self, df: pd.DataFrame) -> pd.DataFrame:
        df['std_flat_type'] = df.apply(lambda x: std_flat_type(x['flat_type'], x['lease_commence_date']), axis=1)
        df['std_flat_model'] = df.apply(lambda x: std_flat_model(x['flat_model']), axis=1)
        df['std_remaining_lease'] = df.apply(lambda x: std_remaining_lease(x['lease_commence_date'], 2023), axis=1)
        df = preprocess_region(df)
        df = preprocess_planning_area(df)
        df = preprocess_flat_model(df)
        df = preprocess_rent_approval_date(df)
        df = df.drop(columns=['block', 'flat_type', 'town', 'subzone', 'lease_commence_date'])
        return df

In [38]:
baseline_truncated_preprocessor = BaselineTruncatedPreprocessor()
baseline_truncated_train = baseline_truncated_preprocessor.fit_transform(raw_train_df)

baseline_truncated_val = baseline_truncated_preprocessor.transform(raw_val_df)
baseline_truncated_test = baseline_truncated_preprocessor.transform(raw_test_df)

In [39]:
baseline_truncated_train.to_csv("data/train/baseline-truncated_train.csv", index=False)
baseline_truncated_val.to_csv("data/train/baseline-truncated_val.csv", index=False)
baseline_truncated_test.to_csv("data/test/baseline-truncated_test.csv", index=False)

In [40]:
baseline_truncated_train.head()

,floor_area_sqm,latitude,longitude,std_flat_type,std_remaining_lease,central region,east region,north region,north-east region,west region,planning_area_ ang mo kio,planning_area_ bedok,planning_area_ bishan,planning_area_ bukit batok,planning_area_ bukit merah,planning_area_ bukit panjang,planning_area_ bukit timah,planning_area_ choa chu kang,planning_area_ clementi,planning_area_ downtown core,planning_area_ geylang,planning_area_ hougang,planning_area_ jurong east,planning_area_ jurong west,planning_area_ kallang,planning_area_ marine parade,planning_area_ novena,planning_area_ outram,planning_area_ pasir ris,planning_area_ punggol,planning_area_ queenstown,planning_area_ rochor,planning_area_ sembawang,planning_area_ sengkang,planning_area_ serangoon,planning_area_ tampines,planning_area_ toa payoh,planning_area_ woodlands,planning_area_ yishun,A,I,NG,OTH,P,S,STD,rent_approval_year,rent_approval_month,monthly_rent
50404,67.0,1.335341,103.854859,3.0,46,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,2021,6,2100
26810,120.0,1.372366,103.880529,5.0,63,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2022,10,2600
19681,67.0,1.335757,103.856790,3.0,46,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,2023,5,2100
7616,68.0,1.347618,103.708644,3.0,90,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2022,3,2400
20363,110.0,1.311570,103.766867,5.0,78,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2023,6,4400


### Dataset 4: Truncated Baseline + Additional Features

In [41]:
class TruncatedFeatEngPreprocessor(BasePreprocessor):
    def __init__(self):
        super().__init__()
        self._coe_prices = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-coe-prices.csv")
        self._amenities_df = pd.read_csv("data/auxiliary-data/auxiliary-data/amenity_data_allmrt.csv")

    def _preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        df = self._clean_data(df)
        df = self._preprocess_baseline_truncate(df)
        df = self._preprocess_feature_eng(df)
        return df

    def _preprocess_baseline_truncate(self, df: pd.DataFrame) -> pd.DataFrame:
        df['std_flat_type'] = df.apply(lambda x: std_flat_type(x['flat_type'], x['lease_commence_date']), axis=1)
        df['std_flat_model'] = df.apply(lambda x: std_flat_model(x['flat_model']), axis=1)
        df['std_remaining_lease'] = df.apply(lambda x: std_remaining_lease(x['lease_commence_date'], 2023), axis=1)
        df = preprocess_region(df)
        df = preprocess_planning_area(df)
        df = preprocess_flat_model(df)
        df = preprocess_rent_approval_date(df)
        df = df.drop(columns=['block', 'flat_type', 'town', 'subzone', 'lease_commence_date'])
        return df

    def _preprocess_feature_eng(self, df: pd.DataFrame) -> pd.DataFrame:
        _, df = compute_mean_coe_prices(self._coe_prices.copy(), df)
        df = count_amenity(df, self._amenities_df.copy(), 0.8)
        return df

In [42]:
truncated_feat_eng_preprocessor = TruncatedFeatEngPreprocessor()
truncated_feat_eng_df_train = truncated_feat_eng_preprocessor.fit_transform(raw_train_df)

truncated_feat_eng_df_val = truncated_feat_eng_preprocessor.transform(raw_val_df)
truncated_feat_eng_df_test = truncated_feat_eng_preprocessor.transform(raw_test_df)

In [43]:
truncated_feat_eng_df_train.to_csv("data/train/truncated-feat-eng_train.csv", index=False)
truncated_feat_eng_df_val.to_csv("data/train/truncated-feat-eng_val.csv", index=False)
truncated_feat_eng_df_test.to_csv("data/test/truncated-feat-eng_test.csv", index=False)

In [44]:
truncated_feat_eng_df_train.head()

,floor_area_sqm,latitude,longitude,std_flat_type,std_remaining_lease,central region,east region,north region,north-east region,west region,planning_area_ ang mo kio,planning_area_ bedok,planning_area_ bishan,planning_area_ bukit batok,planning_area_ bukit merah,planning_area_ bukit panjang,planning_area_ bukit timah,planning_area_ choa chu kang,planning_area_ clementi,planning_area_ downtown core,planning_area_ geylang,planning_area_ hougang,planning_area_ jurong east,planning_area_ jurong west,planning_area_ kallang,planning_area_ marine parade,planning_area_ novena,planning_area_ outram,planning_area_ pasir ris,planning_area_ punggol,planning_area_ queenstown,planning_area_ rochor,planning_area_ sembawang,planning_area_ sengkang,planning_area_ serangoon,planning_area_ tampines,planning_area_ toa payoh,planning_area_ woodlands,planning_area_ yishun,A,I,NG,OTH,P,S,STD,rent_approval_year,rent_approval_month,mean_coe_price,amenity_count,monthly_rent
50404,67.0,1.335341,103.854859,3.0,46,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,2021,6,50698.375,1.0,2100
26810,120.0,1.372366,103.880529,5.0,63,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2022,10,89580.250,2.0,2600
19681,67.0,1.335757,103.856790,3.0,46,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,2023,5,103440.750,4.0,2100
7616,68.0,1.347618,103.708644,3.0,90,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2022,3,79016.000,7.0,2400
20363,110.0,1.311570,103.766867,5.0,78,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2023,6,105592.375,1.0,4400


### Dataset 5: Baseline with Target Encoding of Categorical Variables

In [ ]:
"""
target_encoding_df = cleaned_df
encoder = TargetEncoder(smooth='auto')
target_encoding_df.head()
"""

In [ ]:
"""
target_encoding_df['std_flat_type'] = target_encoding_df.apply(lambda x: std_flat_type(x['flat_type'], x['lease_commence_date']), axis=1)
target_encoding_df['std_flat_model'] = target_encoding_df.apply(lambda x: std_flat_model(x['flat_model']), axis=1)
target_encoding_df['std_remaining_lease'] = target_encoding_df.apply(lambda x: std_remaining_lease(x['lease_commence_date'], 2023), axis=1)

target_encoding_df = preprocess_rent_approval_date(target_encoding_df)
# preprocessed_df = preprocess_region(preprocessed_df)
# preprocessed_df = preprocess_planning_area(preprocessed_df)
# preprocessed_df = preprocess_flat_model(preprocessed_df)

# preprocessed_df = preprocess_town(preprocessed_df)
# preprocessed_df = preprocess_subzone(preprocessed_df)

target_encoding_df['block_num_int'] = target_encoding_df.apply(lambda x: preprocess_block(x['block']), axis=1)
target_encoding_df = target_encoding_df.drop(columns=['block', 'flat_type', 'lease_commence_date', 'flat_model'])
target_encoding_df.head()
"""

In [ ]:
"""
target_encoding_df.head()
"""

In [ ]:
"""
target_encoding_baseline_train, target_encoding_baseline_val = train_test_split(target_encoding_df, test_size=0.1, random_state=42)

categorical_columns = ['town', 'subzone', 'planning_area', 'region', 'std_flat_model']
for col in categorical_columns:
    feats = target_encoding_baseline_train[col].unique()
    for f in feats:
        data = target_encoding_baseline_train.loc[target_encoding_baseline_train[col]==f]
        mean_rent = data['monthly_rent'].mean()
        target_encoding_baseline_train = target_encoding_baseline_train.replace({col: f}, mean_rent)

categorical_columns = ['town', 'subzone', 'planning_area', 'region', 'std_flat_model']
for col in categorical_columns:
    feats = target_encoding_baseline_val[col].unique()
    for f in feats:
        data = target_encoding_baseline_val.loc[target_encoding_baseline_val[col]==f]
        mean_rent = data['monthly_rent'].mean()
        target_encoding_baseline_val = target_encoding_baseline_val.replace({col: f}, mean_rent)


target_encoding_baseline_train.to_csv("data/train/target_encoding_baseline_train.csv", index=False)
target_encoding_baseline_val.to_csv("data/train/target_encoding_baseline_val.csv", index=False)
"""